In [4]:
!pip install pandas
!pip install numpy
!pip install scipy
!pip install seaborn
!pip install matplotlib
!pip install sklearn
!pip install IPython
!pip install torch


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached sklearn-0.0.post11.tar.gz (3.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [18 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  
  If the previous advice does not cover your use case, feel 


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


  Obtaining dependency information for torch from https://files.pythonhosted.org/packages/11/e4/e8a947f36ae2be1223ea8bd8f4714ed6a7a47bf3c6b74705daa17bd127fa/torch-2.1.1-cp310-cp310-win_amd64.whl.metadata
  Obtaining dependency information for filelock from https://files.pythonhosted.org/packages/81/54/84d42a0bee35edba99dee7b59a8d4970eccdd44b99fe728ed912106fc781/filelock-3.13.1-py3-none-any.whl.metadata
  Using cached sympy-1.12-py3-none-any.whl (5.7 MB)
  Obtaining dependency information for networkx from https://files.pythonhosted.org/packages/d5/f0/8fbc882ca80cf077f1b246c0e3c3465f7f415439bdea6b899f6b19f61f70/networkx-3.2.1-py3-none-any.whl.metadata
  Obtaining dependency information for fsspec from https://files.pythonhosted.org/packages/e8/f6/3eccfb530aac90ad1301c582da228e4763f19e719ac8200752a4841b0b2d/fsspec-2023.10.0-py3-none-any.whl.metadata
  Using cached mpmath-1.3.0-py3-none-any.whl (536 kB)
   ---------------------------------------- 0.0/192.3 MB ? eta -:--:--
   ------------


[notice] A new release of pip is available: 23.2.1 -> 23.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
import numpy as np
from scipy.stats import randint
import seaborn as sns # used for plot interactive graph.
import matplotlib.pyplot as plt
import seaborn as sns
from io import StringIO
from sklearn.feature_selection import chi2
from IPython.display import display
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn import metrics
import torch
import torch.nn as nn
import torch.optim as optim
from collections import Counter
import random

In [19]:
import pandas as pd
data = []

# Splitting the file into lines and then into fields
lines = open("train_data.txt", encoding="UTF-8").readlines()
for line in lines:
    if line.strip():  # Checking if the line is not empty
        fields = line.split(' ::: ')
        if len(fields) == 4:  # Ensuring that there are exactly 4 fields
            record = {
                'Title': fields[1],
                'Genre': fields[2],
                'Description': fields[3]
            }
            data.append(record)

# Converting the list of dictionaries to a DataFrame for better visualization
df = pd.DataFrame(data)
df.head()  # Displaying the first few records to verify the parsing


,Title,Genre,Description
0,Oscar et la dame rose (2009),drama,Listening in to a conversation between his doc...
1,Cupid (1997),thriller,A brother and sister with a past incestuous re...
2,"Young, Wild and Wonderful (1980)",adult,As the bus empties the students for their fiel...
3,The Secret Sin (1915),drama,To help their unemployed father make ends meet...
4,The Unrecovered (2007),drama,The film's title refers not only to the un-rec...


In [20]:
df1 = df[['Description', 'Genre']].copy()

In [23]:
import string
df1['Description'] = df1['Description'].apply(lambda x: x.translate(str.maketrans('', '', string.punctuation)))
texts = df1['Description'].tolist()
labels = df1['Genre'].tolist()

# Convert labels to numeric form if they are not already
label_to_ix = {label: i for i, label in enumerate(set(labels))}
numeric_labels = [label_to_ix[label] for label in labels]

# Tokenize texts
tokenized_texts = [text.split() for text in texts]

# Build a vocabulary
vocab = Counter(word for sentence in tokenized_texts for word in sentence)
word_to_ix = {word: i for i, word in enumerate(vocab)}

In [24]:
from torch.utils.data import DataLoader, Dataset

VOCAB_SIZE = len(vocab)
NUM_CLASSES = len(label_to_ix)  # Number of unique labels
EMBEDDING_DIM = 100

class TextDataset(Dataset):
    def __init__(self, texts, labels, word_to_ix):
        self.texts = texts
        self.labels = labels
        self.word_to_ix = word_to_ix

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = self.texts[idx]
        label = self.labels[idx]
        return [self.word_to_ix[word] for word in text], label

def collate_fn(batch):
    texts, labels = zip(*batch)
    lengths = [len(text) for text in texts]
    padded_texts = torch.zeros(len(texts), max(lengths)).long()
    for i, text in enumerate(texts):
        padded_texts[i, :lengths[i]] = torch.LongTensor(text)
    return padded_texts, torch.tensor(labels)


from sklearn.model_selection import train_test_split

# Split data into training and testing sets
train_texts, test_texts, train_labels, test_labels = train_test_split(
    tokenized_texts, numeric_labels, test_size=0.2, random_state=42
)

train_dataset = TextDataset(train_texts, train_labels, word_to_ix)
test_dataset = TextDataset(test_texts, test_labels, word_to_ix)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)
test_loader = DataLoader(test_dataset, batch_size=2, shuffle=True, collate_fn=collate_fn)

In [31]:
class CBOWClassifier(nn.Module):
    def __init__(self, vocab_size, embed_size, num_classes):
        super(CBOWClassifier, self).__init__()
        self.embeddings = nn.Embedding(vocab_size, embed_size)
        self.linear = nn.Linear(embed_size, num_classes)
        
    def forward(self, text):
        embeds = self.embeddings(text)
        embeds_mean = torch.mean(embeds, dim=1)
        out = self.linear(embeds_mean)
        return out

model = CBOWClassifier(VOCAB_SIZE, EMBEDDING_DIM, NUM_CLASSES)
loss_function = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [32]:
EPOCHS = 40  # Adjust as needed

for epoch in range(EPOCHS):
    total_loss = 0
    for context, target in train_loader:
        model.zero_grad()
        log_probs = model(context)
        loss = loss_function(log_probs, target)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch}: Total Loss: {total_loss}")

Epoch 0: Total Loss: 49256.209979474545
Epoch 1: Total Loss: 47358.42084656656
Epoch 2: Total Loss: 46613.65657670796
Epoch 3: Total Loss: 46075.35357300937
Epoch 4: Total Loss: 45598.37224948779
Epoch 5: Total Loss: 45189.57164591551
Epoch 6: Total Loss: 44801.11582966149
Epoch 7: Total Loss: 44541.879960142076
Epoch 8: Total Loss: 44201.99107996002
Epoch 9: Total Loss: 43937.03462521359
Epoch 10: Total Loss: 43684.99571241066
Epoch 11: Total Loss: 43375.37031021714
Epoch 12: Total Loss: 43131.73013336584
Epoch 13: Total Loss: 42891.07268389687
Epoch 14: Total Loss: 42606.57503852621
Epoch 15: Total Loss: 42414.64133260958
Epoch 16: Total Loss: 42198.841249037534
Epoch 17: Total Loss: 42035.442087970674
Epoch 18: Total Loss: 41806.064748775214
Epoch 19: Total Loss: 41584.27425649762
Epoch 20: Total Loss: 41438.51464532316
Epoch 21: Total Loss: 41212.97936266661
Epoch 22: Total Loss: 41028.18850866333
Epoch 23: Total Loss: 40867.46608141251
Epoch 24: Total Loss: 40683.41096935421
Epoch

In [33]:
def calculate_accuracy(model, data_loader):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for context, targets in data_loader:
            outputs = model(context)
            _, predicted = torch.max(outputs, 1)
            total += targets.size(0)
            correct += (predicted == targets).sum().item()
    return correct / total

accuracies = [calculate_accuracy(model, test_loader) for _ in range(10)]
mean_accuracy = np.mean(accuracies)
std_deviation = np.std(accuracies)

print(f"Mean Accuracy: {mean_accuracy}")
print(f"Standard Deviation: {std_deviation}")


Mean Accuracy: 0.47247071843585725
Standard Deviation: 0.0010434112790726519


In [34]:
def get_predictions_and_targets(model, data_loader):
    model.eval()
    all_predictions = []
    all_targets = []

    with torch.no_grad():
        for context, targets in data_loader:
            outputs = model(context)
            _, predicted = torch.max(outputs, 1)
            all_predictions.extend(predicted.tolist())
            all_targets.extend(targets.tolist())

    return all_predictions, all_targets

predictions, targets = get_predictions_and_targets(model, test_loader)


In [35]:
from sklearn.metrics import classification_report

print(classification_report(targets, predictions, target_names=label_to_ix.keys()))


              precision    recall  f1-score   support

     fantasy       0.00      0.00      0.00        74
 documentary       0.61      0.81      0.70      2659
     history       0.00      0.00      0.00        45
       crime       0.00      0.00      0.00       107
        news       0.00      0.00      0.00        34
      horror       0.38      0.26      0.31       431
       sport       0.25      0.03      0.06        93
       short       0.25      0.24      0.25      1045
   talk-show       0.15      0.02      0.04        81
  reality-tv       0.36      0.08      0.14       192
      family       0.00      0.00      0.00       150
       adult       1.00      0.02      0.04       112
    thriller       0.19      0.01      0.02       309
      sci-fi       0.29      0.01      0.03       143
     western       0.62      0.42      0.50       200
   game-show       1.00      0.07      0.14        40
       music       0.34      0.24      0.28       144
      action       0.19    

C:\Users\Nikita\PycharmProjects\nlp_movies\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nikita\PycharmProjects\nlp_movies\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Nikita\PycharmProjects\nlp_movies\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start,